## Optimal Power Flow
**Power Systems Optimization**

by Michael R. Davidson and Jesse D. Jenkins (last updated: September 27, 2020)

This notebook provides an introductory glimpse of the Optimal Power Flow (OPF) problem&mdash;which minimizes the short-run production costs of meeting electricity demand at a number of connected locations from a given set of generators subject to various technical and flow limit constraints. This will be our first treatment of a **network**, which is critical to all power systems.

We will first introduce a model of transmission flows that assumes we can control the flow along each path, in what is called a "transport model". This is a straightforward extension to Economic Dispatch (ED), where we have multiple supply and demand balance constraints, and a new set of flow constraints. This is also similar to other common optimization problems such as fleet routing of shipments.

We will then introduce a linear approximation to the optimal power flow problem known as "DC-OPF", where we begin to incorporate some of the physics involved in how electricity flows along transmission lines. With this, we recognize that given "injections" (i.e., generation) and withdrawals (i.e., demand) of power, flows along lines are not independently controllable. This can (very frequently) result in hitting flow constraints before we would if could control them as in the transport problem.

Our model does not explore the full functionality of DC-OPF, which can include inter-temporal constraints, additional generation constraints (e.g., on voltage), security constraints to ensure stability in the case of contingencies, and network losses.

Full "AC optimal power flow" models are also beyond the scope of this notebook, as the full set of physics associated with the interactions of AC flows introduces non-linearities that are much harder to solve.

We will start off with some simple systems, whose solutions can be worked out manually without resorting to any mathematical optimization model and software. But, eventually we will be solving larger system, thereby emphasizing the importance of such software and mathematical models.

## Introduction to OPF


Optimal Power Flow (OPF) is a power system optimal scheduling problem which fully captures the physics of electricity flows, which adds a leyr of complexity, as well gives a more realistic version of the Economic Dispatch (ED) problem. It usually attempts to capture the entire network topology by representing the interconnections between the different nodes through transmission lines and also representing the electrical parameters, like the resistance, series reactance, shunt admittance etc. of the lines. however, the full-blown "AC" OPF turns out to be an extremely hard problem to solve (usually NP-hard). Hence, system operators and power marketers usually go about solving a linearized version of it, called the DC-OPF. The DC-OPF approximation works satisfactorily for bulk power transmission networks as long as such networks are not operated at the brink of instability or, under very heavily heavily loaded conditions.


## "Transport" model
We will first examine the case where we ignore the physics of electricity flows, and instead treat it like transporting an ordinary commodity.

$$
\begin{align}
\min \ & \sum_{g \in G} VarCost_g \times GEN_g & \\
\text{s.t.} & \\
 & \sum_{g \in G_i} GEN_g - Demand_i = \sum_{j \in J_i} FLOW_{ji} & \forall \quad i \in \mathcal{N}\\
 & FLOW_{ji} \leq MaxFlow_{ji} & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
 & FLOW_{ji} = - FLOW_{ij} & \forall \quad i, j \in \mathcal{N} \\
 & GEN_g \leq Pmax_g & \forall \quad g \in G \\
 & GEN_g \geq Pmin_g & \forall \quad g \in G  
\end{align}
$$

We introduce a few new **sets** in the above:
- $\mathcal{N}$, the set of all nodes (or buses) in the network
- $J_i$, the subset of nodes that are connected to node $i$
- $G_i \subset G$, the subset of generators located at node $i$
 
The **decision variables** in the above problem are:

- $GEN_{g}$, the generation (in MW) produced by each generator, $g$
- $FLOW_{ji}$, the flow (in MW) along the line from $j$ to $i$

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand$, the demand (in MW)
- $MaxFlow_{ji}$, the maximum allowable flow along the line from $j$ to $i$ (0, if no line exists)
- $VarCost_g$, the variable cost of generator $g$

Notice how the problem above is equivalent to producing a single type of good at a set of factories and shipping them along capacity-limited corridors (roads, rail lines, etc.) to meet a set of demands in other locations. 

### 1. Load packages

In [1]:
using JuMP, GLPK
using Plots; plotly();
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/michd/.julia/packages/Plots/ViMfq/src/backends.jl:373


### 2. Load and format data

TO ADD: information on source of data

In [43]:
datadir = joinpath("OPF_data") 
gens = CSV.read(joinpath(datadir,"Gen3.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran3.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load3.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters
lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines2.f
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

# keep only single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand")

,connnode,demand
,Int64,Int64
1,1,-200
2,2,-100
3,3,-100


In [41]:
# This is what the generator dataset looks like:
gens

,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,0,10,0,150,0,150,-150,0,1
2,2,0,10,0,100,0,100,-100,0,2
3,3,0,10,0,150,0,150,-150,0,3


In the above, c2, c1, and c0 are variable cost components of the generators (quadratic, linear and constant, respectively). We will ignore the quadratic and fixed constant costs, leaving:

$$
VarCost_g = c1_g
$$

Here are the transmission lines:

In [21]:
lines

,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id
,Int64,Int64,Float64,Float64,Int64,Int64,Int64
1,1,3,0.01938,0.05917,0,10000,1
2,2,3,0.01938,0.05917,0,10000,2
3,3,1,0.01938,0.05917,0,10000,1
4,3,2,0.01938,0.05917,0,10000,2


We are only using capacity for this model. The dataset also contains resistance and reactance which we will need to calculate losses.

### 3. Create solver function (transport_flow)

In [36]:
#=
Function to solve transport flow problem 
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
Note: it is always a good idea to include a comment blog describing your
function's inputs clearly!
=#
function transport_flow(gens, lines, loads)
    Transport = Model(GLPK.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of all generators
    G = gens.id
    
      # Set of all nodes
    N = union(unique(gens.connnode), 
            unique(loads.connnode))
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Decision variables   
    @variables(Transport, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        FLOW[N,N]        # flow
        # Note: flow is not constrained to be positive
        # By convention, positive will indicate flow from the first to second node
        #  and a negative flow will indicate flow from the second to the first
        # This matrix is thus "anti-symmetric", which we will ensure with an appropriate
        #  constraint
    end)
                
    # Objective function
    @objective(Transport, Min, 
        sum( gens[g,:c0] + gens[g,:c1] * GEN[g] 
                        for g in G)
    )

    # Supply demand balances
    @constraint(Transport, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
                + loads[loads.connnode .== i,:demand][1] ==
        sum(FLOW[j,i] for j in lines[lines.tonode .== i,:fromnode]))

    # Max generation constraint
    @constraint(Transport, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])

    # Flow constraints
    for l in 1:nrow(lines)
        @constraint(Transport, 
            FLOW[lines[l,:fromnode][1],lines[l,:tonode][1]] <= 
                        lines[l,:capacity])
    end
    
    @constraint(Transport, cFlowSymmetric[i in N, j in N],
                    FLOW[j,i] == -FLOW[i,j])

    # Solve statement (! indicates runs in place)
    optimize!(Transport)

    # Dataframe of optimal decision variables
    generation = DataFrame(
        id = gens.id,
        node = gens.connnode,
        gen = value.(GEN).data
        )
    
    flows = value.(FLOW).data

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        flows,
        cost = objective_value(Transport)
    )
end

transport_flow (generic function with 1 method)

In [37]:
solution = transport_flow(gens, lines, loads);
solution.generation

,id,node,gen
,Int64,Int64,Float64
1,1,1,150.0
2,2,2,100.0
3,3,3,150.0


In [38]:
solution.flows

3×3 Array{Float64,2}:
   0.0  0.0  50.0
   0.0  0.0   0.0
 -50.0  0.0   0.0

Hence, the generator located at node 3 (also with id=3) generates 50 MW greater than demand at node 3 and this flows to node 1, which has a total demand of 200. There are no flows into and out of node 2 

## EDIT HERE: DC linear approximation
We will now introduce a linear approximation to the optimal power flow problem that is tractable and reasonably accurate.

$$
\begin{align}
\min \ & \sum_{g \in G} VarCost_g \times GEN_g & \\
\text{s.t.} & \\
 & \sum_{g \in G_i} GEN_g - Demand_i = \sum_{j \in J_i} FLOW_{ji} & \forall \quad i \in \mathcal{N}\\
 & FLOW_{ji} \leq MaxFlow_{ji} & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
 & FLOW_{ji} = - FLOW_{ij} & \forall \quad i, j \in \mathcal{N} \\
 & GEN_g \leq Pmax_g & \forall \quad g \in G \\
 & GEN_g \geq Pmin_g & \forall \quad g \in G  
\end{align}
$$

and:

$$
\begin{align}
		\mathbf{Objective\;Function:}\min_{P_g}\sum_{g\in{G}}C_{g}(P_{g})\longleftarrow\mathbf{power\;generation\; cost}\\
		\mathbf{Subject\;to:\:}{\underline{P}_{g}}\leqslant{P_{g}}\leqslant{{\overline{P}_{g}}},\;\forall{g\in{G}}\longleftarrow\mathbf{MW\; generation\; limits}\\
		P_{g(i)}-P_{d(i)}\longleftarrow\mathbf{real\; power\; injection}\notag\\=\sum_{j\in J(i)}B_{ij}(\theta_j-\theta_i),\;\forall{{i}\in\mathcal{N}}\\
		|P_{ij}|\leqslant{\overline{P}_{ij}},\;\forall{ij}\in{T}\longleftarrow\mathbf{MW\; line\; limit}\\
\end{align}
$$
 
The **decision variable** in the above problem is:

- $P_{g}$, the generation (in MW) produced by each generator, $g$
- $\theta_i$, $\theta_j$ the voltage phase angle of each bus/node, $i,j$

The **parameters** are:

- ${\underline{P}_{g}}$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- ${\overline{P}_{g}}$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $P_{d(i)}$, the demand (in MW) at node $i$
- ${\overline{P}_{ij}}$, the line-flow limit for line connecting buses $i$ and $j$
- $B_{ij}$, susceptance for line connecting buses $i$ and $j$

just like the ED problem, here also, we can safely ignore fixed costs for the purposes of finding optimal dispatch.

In addition, we have:

$$
VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g
$$

With that, let's implement OPF.